# Transmission Line Losses on a Loaded Lossy Line 

When dealing with RF power, for instance in radio, industry or scientific applications, a recurrent problem is to handle the inevitable RF losses correctly to avoid overheating of cables and components. 

## Matched Load
In this example, we will use `scikit-rf` to evaluate the losses in a 50 Ohm, 20 meters long, RG-8 cable (VF=0.84) charged with a resistive load $R_L=50\Omega$ at 13.56 MHz. The cable losses are estimated to 1.483 dB/100 meters and the source power to 400W. 


First, the usual Python imports:

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import skrf as rf

In [ ]:
rf.stylely()

Let's define the problem constants:

In [ ]:
Pin = 400  # W
z0 = 50 # Ohm
freq = rf.Frequency(13.56, npoints=1, unit='MHz')
VF = 0.84
RL = 50  # Ohm
L = 20  # m

The propagation constant of the transmission line $\gamma=\alpha+j\beta$ is: 

In [ ]:
alpha = rf.db_2_np(1.483/100)  # Np/m
beta = freq.w/rf.c/VF
gamma = alpha + 1j*beta

The matched line loss (or power attenuation), $a=e^{2\alpha L}$, is: 

In [ ]:
a = np.exp(2*alpha*L)  # also simply 2.84/100*20
print('Matched line loss: a=', rf.mag_2_db10(a), 'dB')

If the line is charged with a matched load, ie. $R_L=50\Omega$, then the total line loss is $a$. The power lost in the cable will thus be:

In [ ]:
print('(Forward) Power delivered to the  load:', Pin/a, 'W')
print('Power lost in the cable:', Pin *( 1 - 1/a), 'W')

Which can also be checked by the `scikit-rf` transmission line function `zl_2_total_loss`: 

In [ ]:
a_skrf = rf.zl_2_total_loss(z0, zl=RL, theta=gamma*L)
print('Power lost in the cable:',
      Pin * (1 - 1/a_skrf), 'W')

Another way to evaluate the total power dissipated in the circuit is to evaluate the power expression:

$$
P_{delivered} = \frac{1}{2} \Re \left[ V I^* \right]
$$

where $V$ and $I$ are the (peak) total voltage and current. They can be evaluate using the transmission line function `voltage_current_propagation`:

In [ ]:
# reflection coefficient and input impedance
Gamma_in = rf.zl_2_Gamma_in(z0, RL, theta=gamma*L)
Z_in = rf.zl_2_zin(z0, RL, theta=gamma*L)

# voltage and current at the line input as a function of source power 
V_in = np.sqrt(2*z0*Pin)*(1 + Gamma_in)
I_in = V_in/Z_in

# voltage and current at z=L
V,I = rf.voltage_current_propagation(V_in, I_in, z0, gamma*L)
P_delivered = 1/2 * np.real(V * np.conj(I))
print('Power delivered to the load: ', P_delivered, 'W')
print('Power dissipated in the cable: ',Pin - P_delivered, 'W')

## Non Matched load

However, if the load is not perfectly matched to the line characteristic impedance $z_0$, for example with $R_L=200 + 30j\Omega$, additional losses are induced by the reflected wave. The reflection coefficient $\Gamma_{load}$ induced by this load is:

In [ ]:
z0 = 50
ZL = 200 - 30j
Gamma_load = rf.zl_2_Gamma0(z0, ZL)
print('|Gamma_load|=', np.abs(Gamma_load))

while the reflection coefficient seen at the input of the transmission line $\Gamma_{in}$ is: 

In [ ]:
Gamma_in = rf.zl_2_Gamma_in(z0, ZL, theta=gamma*L) 
SWR = rf.Gamma0_2_swr(rf.zl_2_Gamma_in(z0, ZL, theta=gamma*L))
print('|Gamma_in|=', np.abs(Gamma_in), '(SWR=', SWR,')')

The total loss in dB due to SWR is often stated as:
$$
a_{[dB]} + 10 \log_{10} \frac{1 - |\Gamma_{in}|^2}{1 - |\Gamma_{load}|^2}
$$

In [ ]:
10*np.log10(a) + 10*np.log10((1 - np.abs(Gamma_in)**2)/(1 - np.abs(Gamma_load)**2))

However, this expression is only correct if either properties are verified:
- (i) the characteristic impedance of the line is real (distorsionless line) 
- (ii) reflection coefficient is real (ie real $Z_L/Z_0$) [1]. 

The 1st condition is met here, however it will not be the case in the next section. 

Anyway, the `scikit-rf` transmission line function `zl_2_total_loss` is correct in all conditions:

In [ ]:
a = rf.zl_2_total_loss(z0, zl=ZL, theta=gamma * L)
print('Total power loss: ', rf.mag_2_db10(a), 'dB' )
print('Delivered power:', Pin/a, 'W')
print('The total power loss is the cable:', Pin*(1 - 1/a), 'W')

In [ ]:
# reflection coefficient and input impedance
Gamma_in = rf.zl_2_Gamma_in(z0, ZL, theta=gamma*L)
Z_in = rf.zl_2_zin(z0, ZL, theta=gamma*L)

# voltage and current at the line input as a function of source power 
V_in = np.sqrt(2*z0*Pin)*(1 + Gamma_in)
I_in = V_in/Z_in

# voltage and current at z=L
V,I = rf.voltage_current_propagation(V_in, I_in, z0, gamma*L)
P_delivered = 1/2 * np.real(V * np.conj(I))
print('Power delivered to the load: ', P_delivered, 'W')
print('Power dissipated in the cable: ',Pin - P_delivered, 'W')

In [ ]:
rf.Gamma0_2_swr(Gamma_in)

In [ ]:
10*np.log10(P_delivered/Pin)

## A more advanced example
This example reproduces the example presented in reference [1].

Let's assume a coaxial line ([Wireman #551, 450-Ohm](https://thewireman.com/antennap.html)) loaded with a complex load $Z_L=R_L + jX_L$, with the following parameters:  
 - line length: 100 feet
 - frequency: 1.83 MHz
 - [attenuation constant](https://en.wikipedia.org/wiki/Propagation_constant#Attenuation_constant): $\alpha=$ 0.095 dB/100 feet
 - coaxial relative permittivity: $\epsilon_r=1.194$ ([Velocity Factor](https://en.wikipedia.org/wiki/Velocity_factor) VF=0.915)
 - real part of the characteristic impedance: $R_0 = \Re \left[Z_0\right]$=402.75 Ohm
 - Load resistance: $R_L$ = 4.5 Ohm
 - Load reactance: $X_L$ = -1673 Ohm
 

In [ ]:
Z_L = 4.5 - 1673j
R_0 = 402.75
freq = rf.Frequency(1.83, npoints=1, unit='MHz')
VF = 0.915
L = rf.feet_2_meter(100)

First, we can derive the propagation constant $\gamma=\alpha+j\beta$ with $\beta=\frac{\omega}{c }\sqrt{\epsilon_r}$ from the problem parameters:

In [ ]:
alpha = rf.db_2_np(rf.db_per_100feet_2_db_per_100meter(0.095)/100)
beta = freq.w/rf.c/VF
gamma = alpha + 1j*beta
print(gamma)

However, the transmission line characteristic reactance is not given in the problem parameters and must be determined. It can be approximated from a high-frequency, low-loss approximation [1]:
$$
Z_0 = R_0 + j X_0 \approx R_0 - j \frac{\alpha}{\beta}R_0
$$  
ie,
$$
X_0 \approx - \frac{\alpha}{\beta}R_0
$$

In [ ]:
X_0 = -alpha/beta*R_0
Z_0 = R_0 + 1j*X_0 
print('X_0=', X_0)

Now that we have both the characteristic impedance and the propagation constant of the line, the reflection coefficients, input impedance and total loss can be deduced: 

In [ ]:
print('Gamma at load:', np.abs(rf.zl_2_Gamma0(Z_0, Z_L)))
print('Gamma at input:', np.abs(rf.zl_2_Gamma_in(Z_0, Z_L, theta=gamma*L)))
print('SWR at load:', rf.Gamma0_2_swr(rf.zl_2_Gamma0(Z_0, Z_L)))

print('SWR at input:', rf.Gamma0_2_swr(rf.zl_2_Gamma_in(Z_0, Z_L, theta=gamma*L)))
print('Input impedance:', rf.input_impedance_at_theta(Z_0, Z_L, theta=gamma*L ), 'Ohm')

total_loss_db = rf.mag_2_db10(np.abs(rf.zl_2_total_loss(z0=Z_0, zl=Z_L, theta=gamma*L)))
print('Total loss:', total_loss_db, 'dB')

which match the results presented in reference [1].

## References
 - [1] Steve Stearns (K6OIK), A Transmission Line Power Paradox and Its Resolution, ARRL Pacificon Antenna Seminar, Santa Clara, CA          October 10-12, 2014: https://www.fars.k6ya.org/docs/K6OIK-A_Transmission_Line_Power_Paradox_and_Its_Resolution.pdf
 